In [7]:
from pymongo import MongoClient
import json

client = MongoClient('localhost',27017)

db = client['stationdata']

stations=db['stations']

stations.ensure_index([("loc","2dsphere")])

u'loc_2dsphere'

In [4]:
from GTOPO30 import getElevationProfile

help(getElevationProfile)

Help on function getElevationProfile in module GTOPO30:

getElevationProfile(x0, y0, x1, y1)



In [62]:
import re
import numpy as np
from pandas import DataFrame
from GTOPO30 import getElevationProfile

for station in stations.find({
        "_id":re.compile('^KCA')
    }).limit(5):
    # Get neearby stations
    neighborQuery = stations.find({
            "_id":{"$ne":station['_id']},
            "loc":{"$near":{"$geometry":station['loc'],
               "$maxDistance":3000}
              }
        }).limit(5)
    if neighborQuery.count()==0:
        print "Station %s has no neighbors."%station['_id']
    else:
        print "%s's neighbors:"%station['_id']
        for neighbor in neighborQuery:
            c0 = station['loc']['coordinates']
            c1 = neighbor['loc']['coordinates']
            elevProfile = DataFrame(getElevationProfile(c0[0],c0[1],c1[0],c1[1]),columns=('lon','lat','elev'))
            relativePeakHeight = elevProfile['elev'].max() \
                - np.max([
                        elevProfile['elev'][0],
                        elevProfile['elev'][len(elevProfile['elev'])-1]
                    ])
            # TODO:
            #  - check relativePeakHeight against threshold
            #  - check proximity to water
            #  - calc distance
            
            print(neighbor['_id'], elevProfile['elev'].max(),relativePeakHeight)

Station KCAACAMP2 has no neighbors.
KCAACAMP4's neighbors:
(u'KCAACAMP7', 14, 0)
KCAACAMP6's neighbors:
(u'MUP573', 12, 0)
KCAACAMP7's neighbors:
(u'KCAACAMP4', 14, 0)
(u'KCASOUTH8', 14, 0)
(u'KCALODI2', 14, 0)
Station KCAACAMP8 has no neighbors.
